# 1. 복습하기

* ( 피처 스케일링 ) : 서로 다른 변수의 값의 범위를 일정한 수준으로 맞추는 작업으로 ( 정규화 )와 ( 표준화 )가 있습니다.

* ( 정규화 ) : 최소 ~ 최대 범위로 스케일 변환을 하는 것으로 대부분 0 ~ 1 범위에 맞추는 것을 의미합니다. 사이킷런의 정규화를 위한 클래스는? ( MinMaxScaler )
ㄱ
* ( 표준화 ) : 특성의 평균을 0으로 맞추고 분산을 1로 만들어 정규 분포와 같은 특징을 가지도록 합니다. 사이컷런의 표준화를 위한 클래스는? ( StandardScaler )

* 피처 스케일링 시 주의사항은? ( 학습데이터로 fit, 테스트 데이터로는 transform만 )

* ( 로지스틱 회귀 )는 선형 회귀 방식을 기반으로 데이터가 어떤 범주에 속할 확률을 0 ~ 1 사이의 값으로 예측하고 그 확률에 따라 가능성이 더 높은 범주에 속하는 것으로 분류해주는 알고리즘 입니다.

* 로지스틱 회귀는 ( 시그모이드 ) 함수를 이용하여 x값이 아무리 커지거나 작아져도 y값이 항상 0과 1사이의 값이 되도록 반환해 줍니다.

* 사이킷런에서 제공하는 로지스틱 회귀를 위한 클래스는? ( sklearn.linear_model.LogisticRegression )

# 2. 데이터 가져오기

* kaggle.com에서 타이타닉 데이터셋 가져오기 [캐글](https://www.kaggle.com/c/titanic)

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# 데이터 불러오기
import numpy as np
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/titanic/train.csv', index_col='PassengerId')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/titanic/test.csv', index_col='PassengerId')

In [ ]:
# 앞 데이터 보기1
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# 앞 데이터 보기2
test_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# 열 정보 보기1
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [ ]:
# 열 정보 보기2
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


In [ ]:
# Null 건수 보기1
train_df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [ ]:
# Null 건수 보기2
test_df.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# 3. 데이터 전처리하기

* 머신러닝 알고리즘은 Null값과 문자열 값을 입력값으로 허용하지 않습니다.

* Null값 처리하기
  * Null값이 얼마 되지 않는다면 피처의 평균값 또는 중앙값으로 대체할 수 있음
  * Null값이 대부분이라면 해당 피처는 Drop하는 것이 좋음
  * 위의 경우가 아니라면 더 정밀하게 대체값을 선정해야 함

* 범주형 데이터 변환하기 : 학습에 필요한 모든 문자열 값은 인코딩되어 숫자형으로 변환해야 함

* 불필요한 피처 삭제하기 : 예) 단순 아이디와 같은 식별자 피처

## 3-1. Null값 처리하기

In [11]:
# Age : 중앙값으로 채우기
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

In [12]:
# Cabin : 삭제
train_df.drop(['Cabin'], axis=1, inplace=True)
test_df.drop(['Cabin'], axis=1, inplace=True)

In [13]:
# Embarked : 건수 확인하기
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [14]:
# Embarked : 가장 건수가 많은 데이터로 채우기
train_df['Embarked'].fillna('S', inplace=True)
test_df['Embarked'].fillna('S', inplace=True)

In [15]:
# Fare : 중앙값으로 채우기
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

In [16]:
# Null 건수 보기1
train_df.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [17]:
# Null 건수 보기2
test_df.isnull().sum()

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

## 3-2. 범주형 데이터 변환하기

* 머신러닝에서는 학습에 필요한 문자열값은 인코딩되어 수치형 데이터로 변환해 주어야 합니다.

* 만약에 봄을 1, 여름을 2, 가을을 3, 겨울을 4로 변환하게 되면 어떻게 될까요?

* ( 원-핫 인코딩, one-hot encoding ) : 피처 값의 유형에 따라 새로운 피처를 추가하여 고유값에 해당하는 컬럼에만 1을 표시하고 나머지 칼럼에는 0을 표시하는 방식입니다.

* 이렇게 되면 더미로 만든 가변수로 변환하여 관계성이 생기지 않게 됩니다.

In [20]:
# 범주형 데이터 : Pclass, Sex, Embarked
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [19]:
# get_dummies 함수 알아보기
season = pd.DataFrame({
    'season':['spring', 'summer', 'fall', 'winter']
})
season

,season
0,spring
1,summer
2,fall
3,winter


In [28]:
season['season'].str.get_dummies()

,fall,spring,summer,winter
0,0,1,0,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1


In [21]:
pd.get_dummies(season['season'])

,fall,spring,summer,winter
0,0,1,0,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1


In [29]:
# 원-핫 인코딩
train_df = pd.get_dummies(train_df, columns=['Pclass', 'Sex', 'Embarked'])
test_df = pd.get_dummies(test_df, columns=['Pclass', 'Sex', 'Embarked'])
train_df.head()

,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,
1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,0,0,1,0,1,0,0,1
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,1,0,0,1,0,1,0,0
3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,0,0,1,1,0,0,0,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,1,0,0,1,0,0,0,1
5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,0,0,1,0,1,0,0,1


## 3-3. 불필요한 피처 삭제하기

In [30]:
# 불필요한 피처 삭제하기
drop_cols = ['Name', 'Ticket']
train_df.drop(drop_cols, axis=1, inplace=True)
test_df.drop(drop_cols, axis=1, inplace=True)

In [31]:
# 데이터 확인하기1
train_df.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
2,1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
3,1,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
4,1,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
5,0,35.0,0,0,8.0500,0,0,1,0,1,0,0,1


In [32]:
# 데이터 확인하기2
test_df.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,
892,34.5,0,0,7.8292,0,0,1,0,1,0,1,0
893,47.0,1,0,7.0000,0,0,1,1,0,0,0,1
894,62.0,0,0,9.6875,0,1,0,0,1,0,1,0
895,27.0,0,0,8.6625,0,0,1,0,1,0,0,1
896,22.0,1,1,12.2875,0,0,1,1,0,0,0,1


# 4. 데이터 분리하기

In [33]:
# 데이터 분리하기
from sklearn.model_selection import train_test_split

X_train_all = train_df.drop('Survived', axis=1)
y_train_all = train_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.2, random_state=11)
X_train.shape, X_test.shape

((712, 12), (179, 12))

# 5. 모델 선택 및 훈련하기

In [34]:
# 사이킷런 알고리즘 import하기
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [35]:
# Decision Tree
params = {
    'max_depth': [4, 6, 8, 10, 12],
    'min_samples_split': [2, 3, 4]
}
dt = DecisionTreeClassifier(random_state=11)

gs1 = GridSearchCV(dt, param_grid=params, n_jobs=-1)

gs1.fit(X_train, y_train)

print('best parameter:', gs1.best_params_)
print('best score:', gs1.best_score_)
print(gs1.score(X_train, y_train))
print(gs1.score(X_test, y_test))

best parameter: {'max_depth': 6, 'min_samples_split': 4}
best score: 0.8048163104501131
0.8581460674157303
0.8547486033519553


In [36]:
# Random Forest
params = {  
    'max_depth': [4, 6, 8, 10, 12],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [100, 125, 150]
}
rf = RandomForestClassifier(random_state=11)

gs2 = GridSearchCV(rf, param_grid=params, n_jobs=-1)

gs2.fit(X_train, y_train)
 
print('best parameter:', gs2.best_params_)
print('best score:', gs2.best_score_)
print(gs2.score(X_train, y_train))
print(gs2.score(X_test, y_test))

best parameter: {'max_depth': 10, 'min_samples_split': 4, 'n_estimators': 150}
best score: 0.825874125874126
0.925561797752809
0.8547486033519553


In [42]:
# Logistic Regression
params = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 5, 10],
    'max_iter': [500, 750, 1000]
}
lr = LogisticRegression(solver='liblinear')

gs3 = GridSearchCV(lr, param_grid=params, n_jobs=-1)

gs3.fit(X_train, y_train)

print('best parameter:', gs3.best_params_)
print('best score:', gs3.best_score_)
print(gs3.score(X_train, y_train))
print(gs3.score(X_test, y_test))

best parameter: {'C': 0.1, 'max_iter': 500, 'penalty': 'l2'}
best score: 0.7963853048360091
0.7893258426966292
0.8603351955307262


# 6. 모델 예측값 저장하기

In [43]:
# test.csv : 2차원 배열로 변환하기
pred_input = test_df.values
pred_input

array([[34.5,  0. ,  0. , ...,  0. ,  1. ,  0. ],
       [47. ,  1. ,  0. , ...,  0. ,  0. ,  1. ],
       [62. ,  0. ,  0. , ...,  0. ,  1. ,  0. ],
       ...,
       [38.5,  0. ,  0. , ...,  0. ,  0. ,  1. ],
       [27. ,  0. ,  0. , ...,  0. ,  0. ,  1. ],
       [27. ,  1. ,  1. , ...,  1. ,  0. ,  0. ]])

In [44]:
# 모델 예측값 얻기
best_dt = gs1.best_estimator_
pred_dt = best_dt.predict(pred_input)

best_rf = gs2.best_estimator_
pred_rf = best_rf.predict(pred_input)

best_lr = gs3.best_estimator_
pred_lr = best_lr.predict(pred_input)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [49]:
# 파일 저장하기
output = pd.DataFrame({
    'PassengerId': test_df.index,
    'Survived': pred_lr})

output.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/titanic/my_submission.csv', index=False)